# PyTerrier ECIR 2021 Tutorial Notebook - Part 4.2 - ANCE

This notebook provides experiences to attendees in dense indexing and retrieval in [PyTerrier](https://github.com/terrier-org/pyterrier). All experiments are conducted using the [CORD19 corpus](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7251955/) and the [TREC Covid test collection](https://ir.nist.gov/covidSubmit/).

This notebook aims to demonstrate use of the [ANCE dense retrieval](https://github.com/microsoft/ANCE) for end-to-end indexing and retrieval in PyTerrier, as provided by the [*PyTerrier ANCE*](https://github.com/terrierteam/pyterrier_ance) plugin.

In this notebook, you will experience:
 - indexing and retrieval using [*PyTerrier ANCE*](https://github.com/terrierteam/pyterrier_ance).
 - doing a failure analysis comparing ANCE with BM25 results.



# Setup

In the following, we will set up the libraries required to execute the notebook.

## Python packages installation

The following packages are installed to avoid warnings/errors during [PyTerrier](https://github.com/terrier-org/pyterrier) installation. Note that the current release of the [PyTerrier](https://github.com/terrier-org/pyterrier) [ANCE](https://github.com/microsoft/ANCE) plugin works only with the following Python packages:

* `faiss-cpu` or `faiss-gpu`, version 1.6.3

> You can safely ignore the message about runtime restart.

In [ ]:
!apt install --upgrade libomp-dev
!pip install --upgrade faiss-gpu==1.6.3

## Pyterrier installation

The following cell installs the latest release of the [PyTerrier](https://github.com/terrier-org/pyterrier) package.

In [ ]:
!pip install python-terrier

## Pyterrier plugins installation

We install the official version of the [*PyTerrier ANCE*](https://github.com/terrierteam/pyterrier_ance) plugin. You can safely ignore the package versioning errors.

In [ ]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_ance.git

## Trained model download

This downloads the [ANCE](https://github.com/microsoft/ANCE) model checkpoint. We use a University of Glasgow mirror site that results in fasters times than the original ANCE location. Download time can vary - usually about 2 minutes in our experience.

In [ ]:
import os
if not os.path.exists("ance_model_checkpoint.zip"):
    !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/ance_model_checkpoint.zip
    !unzip -j ance_model_checkpoint.zip -d ance_model_checkpoint

# Preliminary steps

## [PyTerrier](https://github.com/terrier-org/pyterrier) initialization

Lets get [PyTerrier](https://github.com/terrier-org/pyterrier) started. This will download the latest version of the [Terrier](http://terrier.org) IR platform.

In [ ]:
import pyterrier as pt

if not pt.started():
    pt.init()

## [TREC-COVID19](https://ir.nist.gov/covidSubmit/) Dataset download

The following cell downloads the [TREC-COVID19](https://ir.nist.gov/covidSubmit/) dataset that we will use in the remainder of this notebook.

In [ ]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='title')
qrels = dataset.get_qrels()

## [Terrier](http://terrier.org) inverted index download

To save a few minutes, we use a pre-built [Terrier](http://terrier.org) inverted index for the [TREC-COVID19](https://ir.nist.gov/covidSubmit/) collection.
Download time took a few seconds for us.

The construction of the inverted index will take few minutes, and the code to use is the following:

```python
import os

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)
```

In [ ]:
import os

if not os.path.exists("terrier_index.zip"):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
  !unzip -j terrier_index.zip -d terrier_index

index_ref = pt.IndexRef.of("./terrier_index/data.properties")
index = pt.IndexFactory.of(index_ref)

## [ANCE](https://github.com/microsoft/ANCE) dense index download

We are going to download a pre-built [ANCE](https://github.com/microsoft/ANCE) [FAISS](https://github.com/facebookresearch/faiss) index for the [TREC-COVID19](https://ir.nist.gov/covidSubmit/) collection. Download time took <1 minute for us.

The construction of this index takes some time. If you wanted to create the index yourself, the code to use is as follows:

```python
!rm -rf /content/anceindex

import pyterrier_ance

ance_indexer = pyterrier_ance.ANCEIndexer(
    checkpoint_path="./ance_model_checkpoint",
    index_path="/content/anceindex",
    num_docs=192509,
    text_attr='abstract' # COVID
)

ance_indexer.index(dataset.get_corpus_iter())
```

In [ ]:
import os

if not os.path.exists("ance_index.zip"):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/ance_index.zip
  !unzip -j ance_index.zip -d ance_index

Lets give a look at the files downloaded.

In [ ]:
!ls -lh /content/ance_index/

So we have a few files that have been generated. In particular, an ANCE index is broken into shards - CORD19 is small enough to only need a single chunk, so we have only `0.*` and no `1.*` etc.

ANCE index contains 3 files:
 - `shards.pkl` - how many documents in each shard;
 - $x$ `.docids.pkl` - docid to docno mapping for documents in each shard;

 - $x$ `.faiss` - the FAISS index for the documents in each shard.


# Retrieval experiments

Now that indexing/downloading has completed, we can load in the index and the learned model (which we will need for encoding queries). Index loading can take some time, as the [FAISS](https://github.com/facebookresearch/faiss) index needs to be loaded in main memory, as well as the docno lookup file.

Lets prepare an experiment. Firstly, lets create in a BM25 baseline transformer, and the [ANCE](https://github.com/microsoft/ANCE) retrieve transformer.

In [ ]:
bm25_retriever = pt.BatchRetrieve(index, wmodel="BM25")

import pyterrier_ance

ance_retriever = pyterrier_ance.ANCERetrieval(
    checkpoint_path="/content/ance_model_checkpoint",
    index_path="/content/ance_index"
)

Now we are ready to run the experiments. We are going to retrieve the top 10 ranked documents for the official topics, and compute several effectiveness metrics. 

In [ ]:
pt.Experiment(
    [bm25_retriever % 10, ance_retriever % 10], 
    topics,
    qrels,
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10", "mrt"],
    names=['BM25', 'ANCE'],
)

## ANCE failure analysis

Lets give a look at the results for BM25 and ANCE for query with qid "`1`". To achieve this, lets make a function that displays the results, including the DOI URL.

In [ ]:
def show_res_with_text_labels(system, qid):
    def make_doi_url(df):
      df["doi"] = df["doi"].apply(lambda doi: "https://doi.org/" + doi)
      return df
    pipe = (system % 10) >> pt.text.get_text(dataset, ["title", "doi"]) >> pt.apply.generic(make_doi_url)
    res = pipe.transform(topics[topics.qid == qid])
    res = res.merge(qrels, how='left')
    def make_clickable(val):
        # target _blank to open new window
        return '<a target="_blank" href="{}">{}</a>'.format(val, val)
    res = res.sort_values("rank", ascending=True)
    res.style.format({'doi': make_clickable})
    return res
  
import pandas as pd
pd.set_option("max_colwidth", 102)

In [ ]:
show_res_with_text_labels(bm25_retriever, "1")

Judging by the titles of the retrieved documents, BM25 did quite well (NB: For documents with DOIs, you can look up the actual document).

Overall, BM25 would have got P@10 = 0.8 for this query. 

Lets look at ANCE now.

In [ ]:
show_res_with_text_labels(ance_retriever, "1")

Here, we see that ANCE has got ranked three documents that had some relevance to the query (titled: "The first two cases of 2019‐nCoV in Italy: Where they come from"), however, only the third one (the one with a DOI) was actually judged relevant. Several other of the retrieved documents appear to be non-relevant, e.g. "General Anesthesia Recommendations for Electroconvulsive Therapy". 

## Practice task

Try a few other queries in `show_res_with_text_labels()`. Qids are numbered from "1" to "50". 

In [ ]:
# Write your code here

#  That's all folks

There are separate notebooks in Part 4 for ColBERT using the Vaswani corpus, as Colab doesnt have sufficient memory for CORD19. Refer to the [tutorial repostiory](https://github.com/terrier-org/ecir2021tutorial) to find the final notebook.

Once you have finished all of the Part 4 notebooks, please dont forget to complete our exit quiz: https://forms.office.com/r/2WbpLiQmWV